# Importation des données

In [12]:
import os
import string
import re
import math
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)

import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle

os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')

with open("data/corpus_trie.file", "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
len(ens_tweets)

1344097

On ne garde ici que les 10 000 premiers tweets

In [13]:
phrases = ens_tweets[0:10000]
words = [item for sublist in phrases for item in sublist]
print(type(words))
vocabulary = set(words)
print("Nombre de mots :", len(words))
print("Taille du vocabulaire :", len(vocabulary))

<class 'list'>
Nombre de mots : 110348
Taille du vocabulaire : 11377


In [54]:
with open("data/input_10k.file", "rb") as f:
    input = pickle.load(f)
with open("data/output_10k.file", "rb") as f:
    output = pickle.load(f)
mat_finale = input+output

# 1. Similarité cosinus
Pour tester la similarité entre deux mots, on calcule la similarité cosinus entre leur représentation vectorielle :
$$
\frac{A \cdot B}{|| A || \cdot || B ||}
$$

In [55]:
mot_poids = {list(vocabulary)[index]: poids.detach().numpy() for (index, poids) in enumerate(mat_finale)}

def cos_distance(u, v):
    return (np.dot(u, v)  / (math.sqrt(np.dot(u, u)) *  (math.sqrt(np.dot(v, v)))))
def mot_plus_proche(word, n=10):
    word_distance = {}
    for mot in mot_poids:
        if mot != word:
            word_distance[mot] = (cos_distance(mot_poids[mot],(mot_poids[word])))
    word_distance = sorted(word_distance.items(), key=lambda t: t[1],reverse=True)
    return word_distance[0:n]

In [61]:
mot_plus_proche("chaud")

[('entendue', 0.7334044060113265),
 ('ouiiiiiii', 0.7236236433768815),
 ('biberon', 0.7194414068781487),
 ('🤦🏻', 0.714956719103805),
 ('☺️', 0.703692035159277),
 ('soudé', 0.6978140989693807),
 ('beaucoup', 0.6899606774649243),
 ('chauffeur', 0.6855752404921275),
 ('lobby', 0.6609641040645329),
 ('what', 0.6602849277376899)]

# 2. Evaluation par le "human judgement"

In [50]:
from scipy.stats import spearmanr
df = pd.read_csv('data_bis/word_similarity.csv', sep=";")
df

,word1,word2,corr
0,corde,sourire,0.00
1,midi,ficelle,0.00
2,coq,périple,0.06
3,fruit,fournaise,0.11
4,autographe,rivage,0.00
...,...,...,...
60,coussin,oreiller,3.00
61,cimetière,cimetière,4.00
62,automobile,auto,3.94
63,joyau,bijou,3.22


In [51]:
def distance_mots(word1,word2):
    if word1 in mot_poids.keys() and word2 in mot_poids.keys():
        word_distance = (cos_distance(mot_poids[word1],(mot_poids[word2])))
    else:
        word_distance = float('nan')
    return word_distance
distance_mots_v = np.vectorize(distance_mots)
df["corr_word2vec"] = distance_mots_v(df["word1"],df["word2"])
print(len(df))
df = df.dropna()
print(len(df))
df

65
7


,word1,word2,corr,corr_word2vec
20,garçon,sage,0.29,0.067709
33,auto,voyage,0.33,0.166322
35,verre,bijou,0.56,-0.159757
38,frère,gars,2.00,-0.262718
53,voyage,périple,2.59,0.222189
59,garçon,gars,3.83,-0.025828
64,midi,dîner,2.17,-0.268673


In [53]:
#On fait des tests à 5 %
alpha = 0.05
corr, p_value = spearmanr(df["corr"], df["corr_word2vec"])
if p_value > alpha:
    print('Le résultat de word2vec est différent de celui du human judgement (non rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)
else:
    print('Le résultat de word2vec est semblable celui du human judgement (rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)

Le résultat de word2vec est différent de celui du human judgement (non rejet de H0 = non corrélation) p=0.819 / Valeur de la corrélation : -0.107
